# Tugas 4


* Lakukan analisa topik modelling dengan menggunakan lsa ( Latent semantic Analysis ) pada data twitter.

* Representasi data dalam bentuk TF-IDF

* Output tugas adalah : bobot kata terhadap masing masing topik bobot setiap topik terhadap  dokumen

* Kata kunci yang digunakan untuk crawling comment youtube capres 2024 

* Tentukan jumlah topik dari 2-10 

* Hasil di upload di repository github ( web jupyterbook)

* materi : https://machinelearninggeek.com/latent-semantic-indexing-using-scikit-learn/


## MEGAMBIL DATASET

* Dataset di ambil dari ulasan Youtube dengan kata kunci Capres 2024
* Komentar di ambil dari akun youtube https://www.youtube.com/watch?v=ox1WDX7MHKQ
* Berikut link crowling data youtube : https://colab.research.google.com/drive/16PbJ_wGN79EtK6J7r-7WXbySobr1UrtT#scrollTo=drJUjdex0P5L

In [ ]:
import pandas as pd 
import numpy as np
# memangggil dataset 
data = pd.read_csv("https://raw.githubusercontent.com/nuskhatulhaqqi/data_mining/main/youtube-comments.csv")
# menampilkan dataset
data.head()

,publishedAt,authorDisplayName,textDisplay,likeCount
0,2023-05-08T06:52:20Z,Mentari Di upuk timur,Survei bohongan untuk menarik simpati rakyat.,0.0
1,2023-05-08T06:34:13Z,Eby Saputra,Pertanyaan nya ini lembaga survei lewat mana m...,0.0
2,2023-05-08T05:47:31Z,Nasrul Nr,"Saya orang Sulsel, mayoritas di sni mendukung ...",0.0
3,2023-05-08T05:46:45Z,Min Chyun,Jangan lupa Bismillah❤,0.0
4,2023-05-08T04:42:03Z,Yoga Reynanda,Prabowo pemimpin terbaik dan sangat cocok jadi...,1.0


In [ ]:
data.shape

(795, 4)

## Pre-Prosessing

### Case Folding

In [ ]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
data['textDisplay'] = data['textDisplay'].str.lower()


print('Case Folding Result : \n')
print(data['textDisplay'].head(5))

Case Folding Result : 

0        survei bohongan untuk menarik simpati rakyat.
1    pertanyaan nya ini lembaga survei lewat mana m...
2    saya orang sulsel, mayoritas di sni mendukung ...
3                               jangan lupa bismillah❤
4    prabowo pemimpin terbaik dan sangat cocok jadi...
Name: textDisplay, dtype: object


### Tokenizing

In [ ]:
# import modul nltk
import nltk
# download paket punkt 
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import string 
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

# ------ Tokenizing ---------

def remove_text_special(text):
    # remove tab, new line, ans back slice
    text = str(text)
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
data['textDisplay'] = data['textDisplay'].apply(remove_text_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

data['textDisplay'] = data['textDisplay'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

data['textDisplay'] = data['textDisplay'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

data['textDisplay'] = data['textDisplay'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

data['textDisplay'] = data['textDisplay'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

data['textDisplay'] = data['textDisplay'].apply(remove_singl_char)

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

data['text_tokens'] = data['textDisplay'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(data['text_tokens'].head())

Tokenizing Result : 

0    [survei, bohongan, untuk, menarik, simpati, ra...
1    [pertanyaan, nya, ini, lembaga, survei, lewat,...
2    [saya, orang, sulsel, mayoritas, di, sni, mend...
3                            [jangan, lupa, bismillah]
4    [prabowo, pemimpin, terbaik, dan, sangat, coco...
Name: text_tokens, dtype: object


### Stopword



In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')

# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])

# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
txt_stopword = pd.read_csv("https://raw.githubusercontent.com/stopwords-iso/stopwords-id/master/stopwords-id.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

data['text_tokens_WSW'] = data['text_tokens'].apply(stopwords_removal) 


print(data['text_tokens_WSW'].head())

0         [survei, bohongan, menarik, simpati, rakyat]
1    [lembaga, survei, mengsurvei, rakyat, persatu,...
2    [orang, sulsel, mayoritas, sni, mendukung, ani...
3                                    [lupa, bismillah]
4    [prabowo, pemimpin, terbaik, cocok, presiden, ...
Name: text_tokens_WSW, dtype: object


### Stemming

In [ ]:
pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 15.1 MB/s eta 0:00:00


In [ ]:
pip install swifter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 830.9/830.9 kB 31.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.1 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.3.4-py3-none-any.whl size=16321 sha256=fb2a459cc691d9ccc3f8edd872e665abdac4a9ca15ffdcc4e1b11a8228e4cb1d
  Stored in directory: /root/.cache/pip/wheels/6c/bd/3e/2d6afc9bc36c9975f8e4215a270bbac6580c4361ebd6bb2323
Successfully built swifter


In [ ]:
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in data['text_tokens_WSW']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

data['text_tokens_stemmed'] = data['text_tokens_WSW'].swifter.apply(get_stemmed_term)
print(data['text_tokens_stemmed'])

2579
------------------------
survei : survei
bohongan : bohong
menarik : tarik
simpati : simpati
rakyat : rakyat
lembaga : lembaga
mengsurvei : mengsurvei
persatu : satu
pilih : pilih
pilpres : pilpres
taunya : tau
unggul : unggul
orang : orang
sulsel : sulsel
mayoritas : mayoritas
sni : sni
mendukung : dukung
anies : anies
tpi : tpi
ganjar : ganjar
kadeer : kadeer
nasdem : nasdem
lupa : lupa
bismillah : bismillah
prabowo : prabowo
pemimpin : pimpin
terbaik : baik
cocok : cocok
presiden : presiden
ri : ri
amatir : amatir
survey : survey
electabilitas : electabilitas
anakanak : anakanak
pa : pa
wakil : wakil
presidensaya : presidensaya
mohon : mohon
berjanji : janji
bawa : bawa
menghidupkan : hidup
fpi : fpi
hti : hti
mendaftar : daftar
kader : kader
pks : pks
independen : independen
perintahkan : perintah
pendukungnya : dukung
adu : adu
domba : domba
menjelek : jelek
jelekkan : jelek
megapdip : megapdip
rendah : rendah
hati : hati
sopan : sopan
santunsuwun : santunsuwun
bagus : bagus


Pandas Apply:   0%|          | 0/795 [00:00<?, ?it/s]

0               [survei, bohong, tarik, simpati, rakyat]
1      [lembaga, survei, mengsurvei, rakyat, satu, pi...
2      [orang, sulsel, mayoritas, sni, dukung, anies,...
3                                      [lupa, bismillah]
4      [prabowo, pimpin, baik, cocok, presiden, ri, a...
                             ...                        
790                     [oyoot, wan, abud, nangisssssss]
791    [kecil, hati, brwalau, survemya, konsisten, br...
792    [oyoot, sabar, bosssmeski, wan, abud, bin, sum...
793    [oyoot, periode, pdip, menang, jokowi, bos, te...
794                            [oyoot, partai, koruptor]
Name: text_tokens_stemmed, Length: 795, dtype: object


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/prosaindata/tugas

/content/drive/MyDrive/prosaindata/tugas


In [ ]:
data.to_csv("Text_Preprocessing-yt.csv")

In [ ]:
data_pre = pd.read_csv("Text_Preprocessing-yt.csv")
data_pre.head()

,Unnamed: 0,publishedAt,authorDisplayName,textDisplay,likeCount,text_tokens,text_tokens_WSW,text_tokens_stemmed
0,0,2023-05-08T06:52:20Z,Mentari Di upuk timur,survei bohongan untuk menarik simpati rakyat,0.0,"['survei', 'bohongan', 'untuk', 'menarik', 'si...","['survei', 'bohongan', 'menarik', 'simpati', '...","['survei', 'bohong', 'tarik', 'simpati', 'raky..."
1,1,2023-05-08T06:34:13Z,Eby Saputra,pertanyaan nya ini lembaga survei lewat mana m...,0.0,"['pertanyaan', 'nya', 'ini', 'lembaga', 'surve...","['lembaga', 'survei', 'mengsurvei', 'rakyat', ...","['lembaga', 'survei', 'mengsurvei', 'rakyat', ..."
2,2,2023-05-08T05:47:31Z,Nasrul Nr,saya orang sulsel mayoritas di sni mendukung a...,0.0,"['saya', 'orang', 'sulsel', 'mayoritas', 'di',...","['orang', 'sulsel', 'mayoritas', 'sni', 'mendu...","['orang', 'sulsel', 'mayoritas', 'sni', 'dukun..."
3,3,2023-05-08T05:46:45Z,Min Chyun,jangan lupa bismillah,0.0,"['jangan', 'lupa', 'bismillah']","['lupa', 'bismillah']","['lupa', 'bismillah']"
4,4,2023-05-08T04:42:03Z,Yoga Reynanda,prabowo pemimpin terbaik dan sangat cocok jadi...,1.0,"['prabowo', 'pemimpin', 'terbaik', 'dan', 'san...","['prabowo', 'pemimpin', 'terbaik', 'cocok', 'p...","['prabowo', 'pimpin', 'baik', 'cocok', 'presid..."


## TF- IDF

### tf idf dengan sklearn

In [ ]:
# mengambil data hasil proses pre-prosessing sebelumnya untuk kategori dan text hasil stemming
datap = pd.read_csv("Text_Preprocessing-yt.csv", usecols=["text_tokens_stemmed"])
datap.columns = ["textDisplay"]
datap

,textDisplay
0,"['survei', 'bohong', 'tarik', 'simpati', 'raky..."
1,"['lembaga', 'survei', 'mengsurvei', 'rakyat', ..."
2,"['orang', 'sulsel', 'mayoritas', 'sni', 'dukun..."
3,"['lupa', 'bismillah']"
4,"['prabowo', 'pimpin', 'baik', 'cocok', 'presid..."
...,...
790,"['oyoot', 'wan', 'abud', 'nangisssssss']"
791,"['kecil', 'hati', 'brwalau', 'survemya', 'kons..."
792,"['oyoot', 'sabar', 'bosssmeski', 'wan', 'abud'..."
793,"['oyoot', 'periode', 'pdip', 'menang', 'jokowi..."


In [ ]:
datap.shape

(795, 1)

In [ ]:
# join list of token as single document string
import ast

def join_text_list(texts):
    texts = ast.literal_eval(texts)
    return ' '.join([text for text in texts])
datap["text_join"] = datap["textDisplay"].apply(join_text_list)

datap["text_join"].head()

0                   survei bohong tarik simpati rakyat
1    lembaga survei mengsurvei rakyat satu pilih pi...
2    orang sulsel mayoritas sni dukung anies tpi du...
3                                       lupa bismillah
4         prabowo pimpin baik cocok presiden ri amatir
Name: text_join, dtype: object

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# banyaknya term yang akan digunakan, 
# di pilih berdasarkan top max_features 
# yang diurutkan berdasarkan term frequency seluruh corpus
max_features = 100000

# Feature Engineering 
print ("------- TF-IDF data -------")

tf_idf = TfidfVectorizer(max_features=max_features, binary=True)
tfidf_mat = tf_idf.fit_transform(datap["text_join"]).toarray()

print("TF-IDF ", type(tfidf_mat), tfidf_mat.shape)

------- TF-IDF data -------
TF-IDF  <class 'numpy.ndarray'> (795, 2246)


In [ ]:
terms = tf_idf.get_feature_names_out() 

# sum tfidf frequency of each term through documents
sums = tfidf_mat.sum(axis=0)

# connecting term to its sums frequency
datas = []
for col, term in enumerate(terms):
    datas.append((term, sums[col] ))

ranking = pd.DataFrame(datas, columns=['term','rank'])
ranking.sort_values('rank', ascending=False)

,term,rank
642,ganjar,61.938983
1613,prabowo,22.732603
1630,presiden,21.077848
1623,pranowo,19.924647
1971,survey,18.456549
...,...,...
119,apbd,0.091879
2064,tidur,0.091879
270,bln,0.091879
755,hilang,0.091879


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import normalize

max_features = 10000

# calc TF vector
cvect = CountVectorizer(max_features=max_features)
TF_vector = cvect.fit_transform(datap["text_join"])

# normalize TF vector
normalized_TF_vector = normalize(TF_vector, norm='l1', axis=1)

# calc IDF
tfidf = TfidfVectorizer(max_features=max_features, smooth_idf=False)
tfs = tfidf.fit_transform(datap["text_join"])
IDF_vector = tfidf.idf_

# hitung TF x IDF sehingga dihasilkan TFIDF matrix / vector
tfidf_mat = normalized_TF_vector.multiply(IDF_vector).toarray()

In [ ]:
# hasil dari tf-idf 
dfbtf= pd.DataFrame(data=tfidf_mat,columns=[terms])
dfbtf

,aalamiin,aamiin,aamin,abal,abalabal,abalquot,abu,abud,abut,aceh,...,yng,yohanes,yohannes,yok,yoqut,yotubbisa,youtubedengan,yra,zaky,zimbabwe
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.471646,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
791,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
792,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.392439,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
793,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Latent Semantic Analysis (LSA)

In [ ]:
from sklearn.decomposition import TruncatedSVD
lsa_model = TruncatedSVD(n_components=10, algorithm='randomized', n_iter=10, random_state=42)

lsa_top=lsa_model.fit_transform(dfbtf)

In [ ]:
print(lsa_top)
print(lsa_top.shape)  # (no_of_doc*no_of_topics)

[[ 2.47210863e-02  1.71268454e-01  6.04513145e-04 ...  4.97723718e-02
  -1.51598060e-01 -2.13294472e-01]
 [ 5.10699132e-02  9.80130484e-02 -4.60445267e-04 ...  3.52919622e-02
  -8.05309702e-02 -1.26331688e-01]
 [ 2.21060662e-01  2.44004477e-02 -7.23980904e-03 ...  1.13846669e-01
   1.40527830e-01 -5.19713783e-02]
 ...
 [ 1.53262700e-02  1.21493322e-03 -6.20700514e-04 ... -1.66735262e-03
  -1.93776396e-03  1.02155047e-04]
 [ 1.95273475e-01 -4.96805044e-04 -5.97456840e-03 ...  1.92758971e-02
   1.43515043e-02 -2.69639475e-03]
 [ 8.93325501e-03  5.22576257e-03  1.61461491e-03 ...  8.20047811e-04
  -1.50131318e-02 -1.70936186e-02]]
(795, 10)


In [ ]:
l=lsa_top[0]
print("Document 0 :")
for i,topic in enumerate(l):
  print("Topic ",i," : ",topic*100)
  

Document 0 :
Topic  0  :  2.472108631454393
Topic  1  :  17.126845416773232
Topic  2  :  0.060451314472458086
Topic  3  :  1.191415464716517
Topic  4  :  2.5773679495329738
Topic  5  :  -7.670676127670134
Topic  6  :  -0.633034477446465
Topic  7  :  4.977237178901279
Topic  8  :  -15.159805958623911
Topic  9  :  -21.329447194582652


In [ ]:
print(lsa_model.components_.shape) # (no_of_topics*no_of_words)
print(lsa_model.components_)

(10, 2246)
[[ 1.48197748e-03  8.67240714e-03  3.07981544e-03 ...  1.51289793e-03
   1.33669159e-04  5.07360623e-04]
 [-6.28034135e-05  7.67342224e-06  2.28239324e-05 ...  4.23300026e-05
   8.21571880e-05  7.70946891e-04]
 [-9.04364534e-05 -7.01827881e-04 -1.92663349e-04 ... -5.11821596e-04
   3.46984170e-06 -7.13810360e-05]
 ...
 [ 1.44980025e-03  1.58239659e-01 -3.23230140e-03 ...  3.82640158e-01
   2.03766841e-04  7.76310225e-03]
 [-1.53304632e-03 -8.65937046e-02 -2.79758810e-03 ... -1.94462898e-01
   2.81886272e-04  9.59852516e-03]
 [ 1.52500780e-04  1.30327058e-02  9.24446228e-04 ...  3.35680876e-02
  -4.11549429e-05 -2.95606600e-03]]


In [ ]:
# most important words for each topic
vocab = tfidf.get_feature_names_out()

for i, comp in enumerate(lsa_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

Topic 0: 
ganjar pranowo ri mantap yes presiden pilih prabowo alhamdulillah joss 

Topic 1: 
abal survey survei anies bayar ha percaya abalabal haram surpai 

Topic 2: 
ora milih nonton suka jijik bokef org barang haranm ogah 

Topic 3: 
amin prabowo presiden anies gp anis bokep cantik menang tugas 

Topic 4: 
ok anies prabowo baswedan presiden alhamdulillah anis yes yess bersih 

Topic 5: 
ok amin ganjar mantap pranowo abal betuuul andiek baswedan joss 

Topic 6: 
alhamdulillah anies bador turun betuuul munafik hahahaha selamat boong jossssspokokepbrgp 

Topic 7: 
amiin yra anies mantap aamiin merah alhamdulillahaamiin survei yes valit 

Topic 8: 
mantap anies wkwkw survey yes ganjarjosss abalabal baswedan santun bersih 

Topic 9: 
abalabal survey betuuul boong amiin kacang ok alhamdulillah jawa bador 



## Latent Dirichlet Allocation (LDA)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=42,max_iter=1) 
# n_components is the number of topics

In [ ]:
lda_top=lda_model.fit_transform(dfbtf)

In [ ]:
print(lda_top.shape)  # (no_of_doc,no_of_topics)
print(lda_top)

(795, 10)
[[0.86068347 0.01548812 0.01547731 ... 0.01547961 0.01547688 0.01547746]
 [0.01591856 0.015892   0.01589533 ... 0.85693632 0.01589544 0.01589326]
 [0.01536827 0.0153678  0.01536786 ... 0.01537033 0.01536844 0.01537048]
 ...
 [0.01324077 0.01324079 0.8808305  ... 0.01324127 0.01324124 0.01324155]
 [0.01598174 0.01596766 0.01597163 ... 0.01597042 0.01596929 0.85626283]
 [0.01512997 0.01512983 0.01513369 ... 0.41900042 0.01512987 0.01513042]]


In [ ]:
sum=0
for i in lda_top[0]:
  sum=sum+i
print(sum)

1.0


In [ ]:
# composition of doc 0 for eg
print("Document 0: ")
for i,topic in enumerate(lda_top[0]):
  print("Topic ",i,": ",topic*100,"%")

Document 0: 
Topic  0 :  86.0683471796724 %
Topic  1 :  1.5488123927457986 %
Topic  2 :  1.5477305941854336 %
Topic  3 :  1.5476736627068066 %
Topic  4 :  1.5476820364066024 %
Topic  5 :  1.5486726206926944 %
Topic  6 :  1.54768740073933 %
Topic  7 :  1.547960866658113 %
Topic  8 :  1.5476876615221795 %
Topic  9 :  1.547745584670632 %


In [ ]:
print(lda_model.components_)
print(lda_model.components_.shape)  # (no_of_topics*no_of_words)

[[0.37997078 0.48180312 0.35821446 ... 0.34279847 0.31053778 0.38779879]
 [0.30822761 0.36592477 0.30790483 ... 0.35044464 0.36294209 0.42442153]
 [0.37215225 0.34095793 0.37476017 ... 0.33801819 0.35551643 0.36384341]
 ...
 [0.35664766 7.19662227 0.34169663 ... 8.11686503 0.36892561 0.39986924]
 [0.3769554  0.37021423 1.40844213 ... 0.35124413 0.85517443 0.34866519]
 [0.36139575 0.36218208 0.36273549 ... 0.34296673 0.38735456 1.4354657 ]]
(10, 2246)


In [ ]:
# most important words for each topic
vocab = tfidf.get_feature_names_out()

for i, comp in enumerate(lda_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

Topic 0: 
survei bayar bador wkwkwk ganjar pesan liat siappp survey beda 

Topic 1: 
ora betuuul abalabal hahahaha preeet pranowo tarik wkwkw indonesia ganjar 

Topic 2: 
anis gin abud wan percaya oyoot nangisssssss jaman modern preed 

Topic 3: 
gunawan menang ganjar ri pranowo yes prabowo penasaran presiden kalah 

Topic 4: 
bravo ganjarjosss jossssspokokepbrgp munafik dillahaamiin jossssbrpokokepgp andiek atas surveibr yaaaa 

Topic 5: 
presiden prabowo amin ganjar koruptor bohong amiin egp mahfud baik 

Topic 6: 
survey bs kampung appah mantulllllllpgpol fitrianaim putar tata survenya bokep 

Topic 7: 
partai pk alhamdulillah yra ok aamiin tugas boong ganjar palsu 

Topic 8: 
abal mark up gaspol nte bener takut anies survey ganjarsalam 

Topic 9: 
ganjar pki mantap no pilih hati anis kadrun baswedan anies 

